# JUICE RPWI HF SID23 (PSSR3 rich): L1a QL -- 2024/7/25

# Import lib

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import copy
import csv
import datetime
import glob

# CDF and Directory setting: set by User

In [ ]:
# The latest CDF library https://spdf.gsfc.nasa.gov/pub/software/cdf/dist/latest/
from spacepy import pycdf

import os
os.environ["CDF_LIB"] = "/Applications/cdf/cdf39_1-dist/lib"

# *** Library ***
sys.path.append('./lib/')
import juice_cdf_lib   as juice_cdf
import juice_math_lib  as juice_math
import juice_sid23_lib as juice_sid23

# Setting and Read CDF file: set by User

In [ ]:
# *** Plot dump ***
dump_mode = 0                           # 0: no-dump  1:plot dump

# *** CAL set ***
band_mode = 0                           # 0: sum    1: /Hz
unit_mode = 0                           # 0: raw    1: dBm＠ADC  2: V@HF   3:V2@HF   4:V2@RWI
cal_mode  = 2                           # [Power]   0: background   1: CAL     2: all
RIME_mode = 1                           # [RIME rejection]  0: n/a  1: RIME reject
RFI_mode  = 1                           # [RFI  rejection]  0: n/a  1: 200kHz reject

# *** Unit set ***
f_mode    = 1                           # [Frequency] 0: linear  1: log
time_mode = 0                           # [Time]      0: Epoch   1: data number

# *** Frenquency range in Linear ***
# f_mode_min = 3000;   f_mode_max = 10000

# *** Parameter set ***
p_raw_max = 8.5                         # background: 7.5   CAL: 10
p_raw_min = 1.0                         # background: 2.5   CAL: 5

# *** Directory set: set by User ***
work_dir = '/Users/user/0-python/JUICE_python/ql/'   # Plot dump folder

In [ ]:
# *** Flight data: Ver.2 ***
"""
# 20240126
data_dir = '/Users/user/0-python/JUICE_data/Data-CDF/2024/01/26/'       # CDF data folder
data_name_list = [#'JUICE_L1a_RPWI-HF-SID23_20240126T092930_V01.cdf',    # 1.5MHz
                  'JUICE_L1a_RPWI-HF-SID23_20240126T094802_V01.cdf',    # 1.5MHz
                  ]
"""
# 202407 -- CO2
data_dir = '/Users/user/Dropbox-Univ/data/data-JUICE/datasets/2024/07/06/'        # CDF data folder
data_name_list = [#'JUICE_L1a_RPWI-HF-SID23_20240706T023200_V01.cdf',
                  #'JUICE_L1a_RPWI-HF-SID23_20240706T124753_V01.cdf',
                  'JUICE_L1a_RPWI-HF-SID23_20240706T130407_V01.cdf',
                 ]

In [ ]:
# *** Ground Test - Ver.2 ***
"""
# 202310 -- SAMPLE
data_dir = '/Users/user/0-python/JUICE_data/test-CCSDS/sample/cdf/'        # CDF data folder
data_name_list = ['SID23_20231024-0011.cdf',
                  'SID23_20231024-0024.cdf',
                  'SID23_20231024-0034.cdf',
                 ]
# 202311 -- FS
data_dir = '/Users/user/0-python/JUICE_data/test-TMIDX/202311_FS/cdf/'        # CDF data folder
data_name_list = ['SID23_Seq07.cdf',
                  'SID23_Seq08.cdf',
                  'SID23_Seq09.cdf',
                 ]
# 202405 -- dryrun check
# TEST
data_dir = '/Users/user/0-python/JUICE_data/CCSDS_test_v2/test_TMIDX/2405_dryrun_test/1_first_parts/'
data_name_list = ['JUICE_L1a_RPWI-HF-SID23_20000108T045452-20000108T045711_V01___TMIDX_00000.bin.cdf',    # 400kHz  pc2_rpwi_pssr3_test_2024_05_15_14_51_46
                  'JUICE_L1a_RPWI-HF-SID23_20000108T045849-20000108T045916_V01___TMIDX_00000.bin.cdf',    # 9MHz    pc2_rpwi_pssr3_9MHz_2024_05_15_14_55_51
                 ]
# LGA
data_dir = '/Users/user/0-python/JUICE_data/CCSDS_test_v2/test_TMIDX/2405_dryrun_test/2_LGA-dryrun/'   
data_name_list = ['JUICE_L1a_RPWI-HF-SID23_20000108T060822-20000108T061002_V01___TMIDX_00000.bin.cdf',    # 9MHz    pc2_LGA-dryrun_RPW-SEQ-08b-1_2024_05_15_16_05_11
                  'JUICE_L1a_RPWI-HF-SID23_20000108T061805-20000108T062005_V01___TMIDX_00000.bin.cdf',    # 9MHz    pc2_LGA-dryrun_RPW-SEQ-08b-2_2024_05_15_16_14_58/TM00
                  'JUICE_L1a_RPWI-HF-SID23_20000108T062025-20000108T062245_V01___TMIDX_00001.bin.cdf',    # 9MHz    pc2_LGA-dryrun_RPW-SEQ-08b-2_2024_05_15_16_14_58/TM01
                  'JUICE_L1a_RPWI-HF-SID23_20000108T062305-20000108T062525_V01___TMIDX_00002.bin.cdf',    # 9MHz    pc2_LGA-dryrun_RPW-SEQ-08b-2_2024_05_15_16_14_58/TM02
                  'JUICE_L1a_RPWI-HF-SID23_20000108T062605-20000108T062725_V01___TMIDX_00003.bin.cdf',    # 9MHz    pc2_LGA-dryrun_RPW-SEQ-08b-2_2024_05_15_16_14_58/TM03
                  'JUICE_L1a_RPWI-HF-SID23_20000108T063854-20000108T064114_V01___TMIDX_00000.bin.cdf',    # 400kHz  pc2_LGA-dryrun_RPW-SEQ-08a-2_2024_05_15_16_35_43/TM00
                  'JUICE_L1a_RPWI-HF-SID23_20000108T064154-20000108T064314_V01___TMIDX_00001.bin.cdf',    # 400kHz  pc2_LGA-dryrun_RPW-SEQ-08a-2_2024_05_15_16_35_43/TM01
                 ]
"""

In [ ]:
# *** Group read
"""
import glob
data_dir = '/Users/user/0-python/JUICE_data/test-CCSDS/sample/cdf/'        # CDF data folder
data_name = 'SID23*.cdf'
cdf_file = data_dir + data_name
data_name_list = glob.glob(cdf_file)
num_list = len(data_name_list)
data_name_list.sort()
for i in range(num_list):
    data_name_list[i] = os.path.split(data_name_list[i])[1]
"""

# get CDF data

In [ ]:
class struct:
    pass

data = struct()
num_list = len(data_name_list)

for i in range(num_list):
    data_name = data_name_list[i]
    cdf_file = data_dir + data_name
    print(i, cdf_file)

    cdf = pycdf.CDF(cdf_file)
    data1 = juice_sid23.hf_sid23_read(cdf)

    if i==0:
        data = data1
        print(data.Eu_i.shape)
    else:
        data = juice_sid23.hf_sid23_add(data, data1)
        print(data.Eu_i.shape)

data_name = os.path.split(data_name)[1];  print("data name:", data_name)

In [ ]:
"""
cdf_file = data_dir + data_name
%ls $cdf_file

cdf = pycdf.CDF(cdf_file)
data = juice_sid23.juice_getdata_hf_sid23(cdf)
"""

In [ ]:
data = juice_sid23.hf_sid23_shaping(data)
date1 = data.epoch[0];  date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[-1]; date2 = date2.strftime('%Y/%m/%d %R:%S')
str_date = date1 + "  -  " + date2;  print(str_date)

n_time0 = data.n_time;  n_block0 = data.time.shape[1];  n_length0 = data.time.shape[2]
n_num0 = n_block0 * n_length0 
print(data.time.shape)

In [ ]:
# Mode 
N_ch0 = data.U_selected[0] + data.V_selected[0] + data.W_selected[0]

print("Ch:", N_ch0, "  (U:", data.U_selected[0], "  V:", data.V_selected[0], "  W:", data.W_selected[0], ")")
print("Num-samples:", n_time0, "   Num-Block:", n_block0, " * Length:", n_length0, " = Num:", n_num0)
print("time_length:", data.time[0][0][n_length0-1], data.time[0][1][0], "sec in 1-block", data.time[0][n_block0-1][n_length0-1], "sec in total")

### RIME rejection ---- "Continuous pulse with Width = 81 (0.28msec)"

In [ ]:
# data = juice_sid23.hf_sid23_rime_reject(data)
if RIME_mode>0:

    data.EuEu = data.Eu_i * data.Eu_i + data.Eu_q * data.Eu_q
    data.EvEv = data.Ev_i * data.Ev_i + data.Ev_q * data.Ev_q
    data.EwEw = data.Ew_i * data.Ew_i + data.Ew_q * data.Ew_q
    data.EE = copy.copy(data.EuEu)
    data.EE_flag = copy.copy(data.EuEu)
    thres = 10

    for i in range(data.n_time):
        data.EE[i] = data.EuEu[i] + data.EvEv[i] + data.EwEw[i]
                    #  * data.U_selected[i] + 

        for j in range(data.N_block[0]):
            EE_med = np.median(data.EE[i][j]);  EE_peak = max(data.EE[i][j])
            index = np.where(data.EE[i][j] <  EE_med * thres);  data.EE_flag[i][j][index] = 0   # EE_med
            index = np.where(data.EE[i][j] >= EE_med * thres);  data.EE_flag[i][j][index] = 1   # EE_peak1 

            # Short off (<40): treated as ON
            k_off = 0
            for k in range(data.N_feed[0]*128):
                if k > 0:
                    if data.EE_flag[i][j][k-1] < data.EE_flag[i][j][k]:     # ON
                        if k - k_off < 40:
                            data.EE_flag[i][j][k_off:k] = data.EE_flag[i][j][k]
                    if data.EE_flag[i][j][k-1] > data.EE_flag[i][j][k]:     # OFF
                        k_off = k

            """
            # Short on (<400: treated as OFF
            for k in range(data.N_feed[0]*128):
                if k > 0:
                    if data.EE_flag[i][j][k-1] > data.EE_flag[i][j][k]:     # off
                        if k - k_on < 40:
                            data.EE_flag[i][j][k_on:k] = data.EE_flag[i][j][k]
                    if data.EE_flag[i][j][k-1] < data.EE_flag[i][j][k]:     # ON
                        k_on = k
            """

            # DIS before/after ON
            k_on = 0
            k_off = 0
            for k in range(data.N_feed[0]*128):
                if k > 0:
                    if data.EE_flag[i][j][k-1] < data.EE_flag[i][j][k]:     # ON
                        k_on = k
                    if data.EE_flag[i][j][k-1] > data.EE_flag[i][j][k]:     # OFF
                        # if i==8 and j==0:
                        #    print(k, k_off, k_on)
                        if k_on -8 > 0:
                            data.EE_flag[i][j][k_on-8:k_on]   = 1
                        if k_off+8 < data.N_feed[0]*128:
                            data.EE_flag[i][j][k_off:k_off+8] = 1
                        k_off = k
            if k_off!=0 and k_off+10 < data.N_feed[0]*128:
                data.EE_flag[i][j][k_off:k_off+10] = 1


    index = np.where(data.EE_flag > 0)
    # data.EE[index] = EE_med
    data.Eu_i2 = copy.copy(data.Eu_i);  data.Eu_q2 = copy.copy(data.Eu_q)
    data.Ev_i2 = copy.copy(data.Ev_i);  data.Ev_q2 = copy.copy(data.Ev_q)
    data.Ew_i2 = copy.copy(data.Ew_i);  data.Ew_q2 = copy.copy(data.Ew_q)
    data.Eu_i2[index] = 0;  data.Eu_q2[index] = 0
    data.Ev_i2[index] = 0;  data.Ev_q2[index] = 0
    data.Ew_i2[index] = 0;  data.Ew_q2[index] = 0
    data.Eu_i[index] = 0;  data.Eu_q[index] = 0
    data.Ev_i[index] = 0;  data.Ev_q[index] = 0
    data.Ew_i[index] = 0;  data.Ew_q[index] = 0

    #
    # ---- TEST PLOT ----
    #
    n_sweep = 1
    n_block = 9 # n_block0
    print("[specific block]  SWEEP:", n_sweep, "in total of ", n_time0, "   Block:", n_block, "in total of ", data.N_block[0])

    fig = plt.figure(figsize=(14, 14))
    ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4)

    """
    ax1.plot(np.ravel(data.EE_flag[n_sweep][n_block]), '-k', linewidth=1,  label='EE_flag');  # TMP
    ax1.plot(np.ravel(data.EE_flag[n_sweep][n_block]), '-k', linewidth=1,  label='EE_flag');  # TMP
    ax1.plot(np.ravel(data.EE     [n_sweep][n_block]), '-r', linewidth=1,  label='EE');          # TMP
    ax2.plot(np.ravel(data.Eu_i2  [n_sweep][n_block]), '-b', linewidth=.5, label='Eu_i'); 
    ax2.plot(np.ravel(data.Eu_q2  [n_sweep][n_block]), '-r', linewidth=.5, label='Eu_q')
    ax3.plot(np.ravel(data.Ev_i2  [n_sweep][n_block]), '-b', linewidth=.5, label='Ev_i'); 
    ax3.plot(np.ravel(data.Ev_q2  [n_sweep][n_block]), '-r', linewidth=.5, label='Ev_q')
    ax4.plot(np.ravel(data.Ew_i2  [n_sweep][n_block]), '-b', linewidth=.5, label='Ew_i'); 
    ax4.plot(np.ravel(data.Ew_q2  [n_sweep][n_block]), '-r', linewidth=.5, label='Ew_q')
    """
    """
    ax1.plot(np.ravel(data.EE_flag[n_sweep]), '-k', linewidth=1,  label='EE_flag');  # TMP
    ax1.plot(np.ravel(data.EE_flag[n_sweep]), '-k', linewidth=1,  label='EE_flag');  # TMP
    ax1.plot(np.ravel(data.EE     [n_sweep]), '-r', linewidth=1,  label='EE');          # TMP
    ax2.plot(np.ravel(data.Eu_i2  [n_sweep]), '-b', linewidth=.5, label='Eu_i'); 
    ax2.plot(np.ravel(data.Eu_q2  [n_sweep]), '-r', linewidth=.5, label='Eu_q')
    ax3.plot(np.ravel(data.Ev_i2  [n_sweep]), '-b', linewidth=.5, label='Ev_i'); 
    ax3.plot(np.ravel(data.Ev_q2  [n_sweep]), '-r', linewidth=.5, label='Ev_q')
    ax4.plot(np.ravel(data.Ew_i2  [n_sweep]), '-b', linewidth=.5, label='Ew_i'); 
    ax4.plot(np.ravel(data.Ew_q2  [n_sweep]), '-r', linewidth=.5, label='Ew_q')
    """
    ax1.plot(np.ravel(data.EE_flag), '-k', linewidth=1,  label='EE_flag');  # TMP
    ax1.plot(np.ravel(data.EE     ), '-r', linewidth=1,  label='EE');          # TMP
    ax2.plot(np.ravel(data.Eu_i2  ), '-b', linewidth=.5, label='Eu_i'); 
    ax2.plot(np.ravel(data.Eu_q2  ), '-r', linewidth=.5, label='Eu_q')
    ax3.plot(np.ravel(data.Ev_i2  ), '-b', linewidth=.5, label='Ev_i'); 
    ax3.plot(np.ravel(data.Ev_q2  ), '-r', linewidth=.5, label='Ev_q')
    ax4.plot(np.ravel(data.Ew_i2  ), '-b', linewidth=.5, label='Ew_i'); 
    ax4.plot(np.ravel(data.Ew_q2  ), '-r', linewidth=.5, label='Ew_q')

    ax2.set_ylabel('Eu');  ax3.set_ylabel('Ev');  ax4.set_ylabel('Ew')
    ax4.set_xlabel('Time (sec)')
    #
    title_label = '[JUICE/RPWI HF PSSR3-Rich (SID-23)]  ' + data_name + "  [" + date1 + "]";  ax1.set_title(title_label)
    ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8);  ax4.legend(loc='upper right', fontsize=8)

    # range: X-axis
    """
    # xlim=[0.00159, 0.00187]   # 0.28msec
    xlim=[1300, 1420]             # 81 * 3.37837832e-06 = 0.28msec
    ax1.set_xlim(xlim);  ax2.set_xlim(xlim);  ax3.set_xlim(xlim);  ax4.set_xlim(xlim)
    # range: Y-axis
    ylim=[-10**(p_raw_max-3.5), 10**(p_raw_max-3.5)]
    ax1.set_ylim(ylim);  ax2.set_ylim(ylim);  ax3.set_ylim(ylim);  ax4.set_ylim(ylim)
    """
    ax1.set_yscale('log')

### RFI noise rejection

In [ ]:
# data = juice_sid23.hf_sid23_rfi_reject(data)
if RFI_mode > 0:

    from scipy import signal
    dt = 1.0 / juice_cdf._sample_rate(data.decimation_AUX[0])
    f_cut = 5000                   # Cutoff: 10kHz
    f_tap = 129
    filter = signal.firwin(numtaps=f_tap, cutoff=f_cut, fs=1/dt, pass_zero=False)

    print("frequency cutoff(Hz):", f_cut, "   delay:", dt * f_tap )
    for i in range(data.n_time):
        for j in range(data.N_block[0]):
            data.Eu_i[i][j] = signal.lfilter(filter, 1, data.Eu_i[i][j])
            data.Eu_q[i][j] = signal.lfilter(filter, 1, data.Eu_q[i][j])
            data.Ev_i[i][j] = signal.lfilter(filter, 1, data.Ev_i[i][j])
            data.Ev_q[i][j] = signal.lfilter(filter, 1, data.Ev_q[i][j])
            # data.Ew_i[i][j] = signal.lfilter(filter, 1, data.Ew_i[i][j])
            # data.Ew_q[i][j] = signal.lfilter(filter, 1, data.Ew_q[i][j])

## CAL Waveform

In [ ]:
# *** Conversion factor: cal_mode ***
cf, p_max0, p_min0, str_unit = juice_cdf.cal_factors(unit_mode, p_raw_max, p_raw_min)
print("conversion factor:", cf, "    MAX-min:", p_max0, p_min0, "   str_unit:", str_unit)

# Raw data

In [ ]:
fig = plt.figure(figsize=(14, 16))
ax1 = fig.add_subplot(6, 1, 1);  ax2 = fig.add_subplot(6, 1, 2);  ax3 = fig.add_subplot(6, 1, 3);  ax4 = fig.add_subplot(6, 1, 4)
ax5 = fig.add_subplot(6, 1, 5);  ax6 = fig.add_subplot(6, 1, 6);  

ax1.plot(np.ravel(data.Eu_i[:][:]), '-b', linewidth=.5, label='Eu_i'); ax1.plot(np.ravel(data.Eu_q[:][:]), ':r', linewidth=.1, label='Eu_q')
ax2.plot(np.ravel(data.Ev_i[:][:]), '-b', linewidth=.5, label='Ev_i'); ax2.plot(np.ravel(data.Ev_q[:][:]), ':r', linewidth=.1, label='Ev_q')
ax3.plot(np.ravel(data.Ew_i[:][:]), '-b', linewidth=.5, label='Ew_i'); ax3.plot(np.ravel(data.Ew_q[:][:]), ':r', linewidth=.1, label='Ew_q')
ax4.plot(np.ravel(data.time[:]), linewidth=.5, label='time')
ax5.plot(np.ravel(data.freq_center[:]), '.')
ax6.plot(np.ravel(data.epoch[:]), '.')
ax1.set_ylabel('Eu');  ax2.set_ylabel('Ev');  ax3.set_ylabel('Ew');   
ax4.set_ylabel('Time [sec]');  ax5.set_ylabel('f_center [kHz]');  ax6.set_xlabel(str_date)
#
title_label = '[JUICE/RPWI HF PSSR3-Rich (SID-23)]  ' + data_name;  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8)
ax4.legend(loc='upper right', fontsize=8)

# range: X-axis
xlim=[0, len(np.ravel(data.Eu_i[:][:]))];  print(xlim)
ax1.set_xlim(xlim);  ax2.set_xlim(xlim);  ax3.set_xlim(xlim);  ax4.set_xlim(xlim)
xlim=[0, len(np.ravel(data.epoch[:]))];    print(xlim)
ax5.set_xlim(xlim); ax6.set_xlim(xlim)
"""
# range: Y-axis
ylim=[-10**(p_raw_max-3.5), 10**(p_raw_max-3.5)]
ax1.set_ylim(ylim);  ax3.set_ylim(ylim);  ax5.set_ylim(ylim)
"""

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw.png'
    fig.savefig(png_fname)

## RAW - 1 Sweep

In [ ]:
n_sweep = np.int16(n_time0/2)
# n_sweep = n_time0-4
n_block = n_block0
print("[specific sweep]  SWEEP:", n_sweep, "in total of", n_time0, "   Block:", n_block, "in total of", data.N_block[0])

fig = plt.figure(figsize=(12, 10))
ax1 = fig.add_subplot(4, 1, 1);  ax3 = fig.add_subplot(4, 1, 2);  ax5 = fig.add_subplot(4, 1, 3);  ax7 = fig.add_subplot(4, 1, 4);  

ax1.plot(np.ravel(data.Eu_i[n_sweep][0:n_block]), '-b', linewidth=.5, label='Eu_i'); 
ax1.plot(np.ravel(data.Eu_q[n_sweep][0:n_block]), ':r', linewidth=.5, label='Eu_q')
ax3.plot(np.ravel(data.Ev_i[n_sweep][0:n_block]), '-b', linewidth=.5, label='Ev_i'); 
ax3.plot(np.ravel(data.Ev_q[n_sweep][0:n_block]), ':r', linewidth=.5, label='Ev_q')
ax5.plot(np.ravel(data.Ew_i[n_sweep][0:n_block]), '-b', linewidth=.5, label='Ew_i'); 
ax5.plot(np.ravel(data.Ew_q[n_sweep][0:n_block]), ':r', linewidth=.5, label='Ew_q')
ax7.plot(np.ravel(data.time[n_sweep][0:n_block]), linewidth=.5, label='time')
ax1.set_ylabel('Eu');  ax3.set_ylabel('Ev');  ax5.set_ylabel('Ew');    ax7.set_ylabel('Time [sec]')
#
title_label = '[JUICE/RPWI HF PSSR3-Rich (SID-23)]  ' + data_name + "  [" + date1 + "]";  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8);  ax5.legend(loc='upper right', fontsize=8)
ax7.legend(loc='upper right', fontsize=8)

# range: X-axis
"""
xlim=[0, n_block*data.N_feed[0]*128]
ax1.set_xlim(xlim);  ax3.set_xlim(xlim);  ax5.set_xlim(xlim);  ax7.set_xlim(xlim)
"""
# range: Y-axis
"""
ylim=[-10**(p_raw_max-3.5), 10**(p_raw_max-3.5)]
ax1.set_ylim(ylim);  ax3.set_ylim(ylim);  ax5.set_ylim(ylim)
"""

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw-sweep.png'
    fig.savefig(png_fname)

## RAW - 1 block

In [ ]:
n_sweep = np.int16(n_time0/2)
n_block = 1
print("[specific block]  SWEEP:", n_sweep, "in total of ", n_time0, "   Block:", n_block, "in total of ", data.N_block[0])

fig = plt.figure(figsize=(12, 10))
ax1 = fig.add_subplot(3, 1, 1);  ax3 = fig.add_subplot(3, 1, 2);  ax5 = fig.add_subplot(3, 1, 3)

ax1.plot(np.ravel(data.time[n_sweep][0:n_block]), np.ravel(data.Eu_i[n_sweep][0:n_block]), '-b', linewidth=.5, label='Eu_i'); 
ax1.plot(np.ravel(data.time[n_sweep][0:n_block]), np.ravel(data.Eu_q[n_sweep][0:n_block]), '-r', linewidth=.5, label='Eu_q')
ax3.plot(np.ravel(data.time[n_sweep][0:n_block]), np.ravel(data.Ev_i[n_sweep][0:n_block]), '-b', linewidth=.5, label='Ev_i'); 
ax3.plot(np.ravel(data.time[n_sweep][0:n_block]), np.ravel(data.Ev_q[n_sweep][0:n_block]), '-r', linewidth=.5, label='Ev_q')
ax5.plot(np.ravel(data.time[n_sweep][0:n_block]), np.ravel(data.Ew_i[n_sweep][0:n_block]), '-b', linewidth=.5, label='Ew_i'); 
ax5.plot(np.ravel(data.time[n_sweep][0:n_block]), np.ravel(data.Ew_q[n_sweep][0:n_block]), '-r', linewidth=.5, label='Ew_q')
ax1.set_ylabel('Eu');  ax3.set_ylabel('Ev');  ax5.set_ylabel('Ew')
ax5.set_xlabel('Time (sec)')
#
title_label = '[JUICE/RPWI HF PSSR3-Rich (SID-23)]  ' + data_name + "  [" + date1 + "]";  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8);  ax5.legend(loc='upper right', fontsize=8)

# range: X-axis
"""
# xlim=[0.00159, 0.00187]   # 0.28msec
xlim=[334, 415]             # 81 * 3.37837832e-06 = 0.28msec
ax1.set_xlim(xlim);  ax3.set_xlim(xlim);  ax5.set_xlim(xlim);  ax7.set_xlim(xlim)
# range: Y-axis
ylim=[-10**(p_raw_max-3.5), 10**(p_raw_max-3.5)]
ax1.set_ylim(ylim);  ax3.set_ylim(ylim);  ax5.set_ylim(ylim)
"""

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw-block.png'
    fig.savefig(png_fname)

In [ ]:
print("N_samp:", data.N_samp[0], "   decimation:", juice_cdf._sample_rate(data.decimation[0])) 
peak_E = np.ravel(data.Eu_i[:][:]); peak_f = np.argmax(peak_E); print("Peak Eu_i:", '{:.2e}'.format(peak_E[peak_f]))
peak_E = np.ravel(data.Ev_i[:][:]); peak_f = np.argmax(peak_E); print("Peak Ev_i:", '{:.2e}'.format(peak_E[peak_f]))
peak_E = np.ravel(data.Ew_i[:][:]); peak_f = np.argmax(peak_E); print("Peak Ew_i:", '{:.2e}'.format(peak_E[peak_f]))
peak_E = np.ravel(data.Eu_q[:][:]); peak_f = np.argmax(peak_E); print("Peak Eu_q:", '{:.2e}'.format(peak_E[peak_f]))
peak_E = np.ravel(data.Ev_q[:][:]); peak_f = np.argmax(peak_E); print("Peak Ev_q:", '{:.2e}'.format(peak_E[peak_f]))
peak_E = np.ravel(data.Ew_q[:][:]); peak_f = np.argmax(peak_E); print("Peak Ew_q:", '{:.2e}'.format(peak_E[peak_f]))

# Get Spectrum

In [ ]:
# Spec mode:           0: low resolution  1; high resolution  
spec = juice_sid23.hf_sid23_getspec(data, band_mode)
power_str = juice_cdf.power_label(band_mode, unit_mode)
#
f_min = spec.freq[0];  f_max = spec.freq[n_length0-1]
d_freq = spec.freq[1]-spec.freq[0]

print(spec.EE.shape, n_time0, n_block0, n_length0)
print(f_min, "kHz -", f_max, "kHz  (Center:", data.freq_center[0], "kHz  df:", d_freq, "kHz")

print("f_center:", data.freq_center[0], "  df:", spec.freq[1]-spec.freq[0])
print(data.freq_center)

# Spectrum - first

In [ ]:
n_sweep = 0
p_min = p_raw_min-4;  p_max = p_raw_max

fig = plt.figure(figsize=(12, 10))
ax1 = fig.add_subplot(4, 1, 1);  ax4 = fig.add_subplot(4, 1, 2);  ax5 = fig.add_subplot(4, 1, 3);  ax6 = fig.add_subplot(4, 1, 4)

ax1.plot(spec.freq, np.ravel(spec.EE[n_sweep][0][:]), '-r', linewidth=.5, label='EuEu')
ax4.plot(spec.freq, np.ravel(spec.EuEu[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax5.plot(spec.freq, np.ravel(spec.EvEv[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax6.plot(spec.freq, np.ravel(spec.EwEw[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax1.plot(spec.freq, np.ravel(spec.EE[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='EvEv')
ax4.plot(spec.freq, np.ravel(spec.EuEu[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='half')
ax5.plot(spec.freq, np.ravel(spec.EvEv[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='half')
ax6.plot(spec.freq, np.ravel(spec.EwEw[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='half')
ax1.plot(spec.freq, np.ravel(spec.EE[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='EwEw')
ax4.plot(spec.freq, np.ravel(spec.EuEu[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax5.plot(spec.freq, np.ravel(spec.EvEv[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax6.plot(spec.freq, np.ravel(spec.EwEw[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax1.set_ylabel('EE');  ax4.set_ylabel('EuEu'); ax5.set_ylabel('EvEv'); ax6.set_ylabel('EwEw'); ax6.set_xlabel('Frequency [kHz]')
ax1.set_yscale('log'); ax4.set_yscale('log');  ax5.set_yscale('log');  ax6.set_yscale('log')
#
date1 = data.epoch[n_sweep];  date1 = date1.strftime('First: %Y/%m/%d %R:%S')
title_date = "[" + data_name + "]  " + date1;  ax1.set_title(title_date)
#
ax1.legend(loc='upper right', fontsize=8);  
ax4.legend(loc='upper right', fontsize=8);  ax5.legend(loc='upper right', fontsize=8);   ax6.legend(loc='upper right', fontsize=8)

# range: Y-axis
ylim=[10**(p_min), 10**(p_max)]
ax1.set_ylim(ylim); ax4.set_ylim(ylim);  ax5.set_ylim(ylim); ax6.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec-1st.png'
    fig.savefig(png_fname)

# Peak
print("[First]")
peak_E = np.ravel(spec.EE[n_sweep][0]);   peak_f = np.argmax(peak_E); print("Peak EE  :", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(spec.freq[peak_f]), "kHz")
peak_E = np.ravel(spec.EuEu[n_sweep][0]); peak_f = np.argmax(peak_E); print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(spec.freq[peak_f]), "kHz")
peak_E = np.ravel(spec.EvEv[n_sweep][0]); peak_f = np.argmax(peak_E); print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(spec.freq[peak_f]), "kHz")
peak_E = np.ravel(spec.EwEw[n_sweep][0]); peak_f = np.argmax(peak_E); print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(spec.freq[peak_f]), "kHz")

## Spectrum - Mid

In [ ]:
n_sweep = np.int16(n_time0/2)
# n_sweep = n_time0-3
print("[mid]  SWEEP:", n_num0, "in total of", n_time0)
p_min = p_raw_min-4;  p_max = p_raw_max

fig = plt.figure(figsize=(12, 10))
ax1 = fig.add_subplot(4, 1, 1);  ax4 = fig.add_subplot(4, 1, 2);  ax5 = fig.add_subplot(4, 1, 3);  ax6 = fig.add_subplot(4, 1, 4)

ax1.plot(spec.freq, np.ravel(spec.EE[n_sweep][0][:]), '-r', linewidth=.5, label='EuEu')
ax4.plot(spec.freq, np.ravel(spec.EuEu[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax5.plot(spec.freq, np.ravel(spec.EvEv[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax6.plot(spec.freq, np.ravel(spec.EwEw[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax1.plot(spec.freq, np.ravel(spec.EE[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='EvEv')
ax4.plot(spec.freq, np.ravel(spec.EuEu[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='half')
ax5.plot(spec.freq, np.ravel(spec.EvEv[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='half')
ax6.plot(spec.freq, np.ravel(spec.EwEw[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='half')
ax1.plot(spec.freq, np.ravel(spec.EE[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='EwEw')
ax4.plot(spec.freq, np.ravel(spec.EuEu[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax5.plot(spec.freq, np.ravel(spec.EvEv[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax6.plot(spec.freq, np.ravel(spec.EwEw[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax1.set_ylabel('EE');  ax4.set_ylabel('EuEu'); ax5.set_ylabel('EvEv'); ax6.set_ylabel('EwEw'); ax6.set_xlabel('Frequency [kHz]')
ax1.set_yscale('log'); ax4.set_yscale('log');  ax5.set_yscale('log');  ax6.set_yscale('log')
#
date1 = data.epoch[n_sweep];  date1 = date1.strftime('Mid: %Y/%m/%d %R:%S')
title_date = "[" + data_name + "]  " + date1;  ax1.set_title(title_date)
#
ax1.legend(loc='upper right', fontsize=8);  
ax4.legend(loc='upper right', fontsize=8);  ax5.legend(loc='upper right', fontsize=8);   ax6.legend(loc='upper right', fontsize=8)

# range: Y-axis
ylim=[10**(p_min), 10**(p_max)]
ax1.set_ylim(ylim);  ax4.set_ylim(ylim);  ax5.set_ylim(ylim);  ax6.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec-mid.png'
    fig.savefig(png_fname)

# Peak
print("[Mid]")
peak_E = np.ravel(spec.EE[n_sweep][0]);   peak_f = np.argmax(peak_E); print("Peak EE  :", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(spec.freq[peak_f]), "kHz")
peak_E = np.ravel(spec.EuEu[n_sweep][0]); peak_f = np.argmax(peak_E); print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(spec.freq[peak_f]), "kHz")
peak_E = np.ravel(spec.EvEv[n_sweep][0]); peak_f = np.argmax(peak_E); print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(spec.freq[peak_f]), "kHz")
peak_E = np.ravel(spec.EwEw[n_sweep][0]); peak_f = np.argmax(peak_E); print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(spec.freq[peak_f]), "kHz")

## Spectrum - Last

In [ ]:
n_sweep = n_time0-1
p_min = p_raw_min-2;  p_max = p_raw_max-2

fig = plt.figure(figsize=(12, 10))
ax1 = fig.add_subplot(4, 1, 1);  ax4 = fig.add_subplot(4, 1, 2);  ax5 = fig.add_subplot(4, 1, 3);  ax6 = fig.add_subplot(4, 1, 4)

ax1.plot(spec.freq, np.ravel(spec.EE[n_sweep][0][:]), '-r', linewidth=.5, label='EuEu')
ax4.plot(spec.freq, np.ravel(spec.EuEu[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax5.plot(spec.freq, np.ravel(spec.EvEv[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax6.plot(spec.freq, np.ravel(spec.EwEw[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax1.plot(spec.freq, np.ravel(spec.EE[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='EvEv')
ax4.plot(spec.freq, np.ravel(spec.EuEu[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='half')
ax5.plot(spec.freq, np.ravel(spec.EvEv[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='half')
ax6.plot(spec.freq, np.ravel(spec.EwEw[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='half')
ax1.plot(spec.freq, np.ravel(spec.EE[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='EwEw')
ax4.plot(spec.freq, np.ravel(spec.EuEu[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax5.plot(spec.freq, np.ravel(spec.EvEv[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax6.plot(spec.freq, np.ravel(spec.EwEw[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax1.set_ylabel('EE');  ax4.set_ylabel('EuEu'); ax5.set_ylabel('EvEv'); ax6.set_ylabel('EwEw'); ax6.set_xlabel('Frequency [kHz]')
ax1.set_yscale('log'); ax4.set_yscale('log');  ax5.set_yscale('log');  ax6.set_yscale('log')
#
date1 = data.epoch[n_sweep];  date1 = date1.strftime('Last: %Y/%m/%d %R:%S')
title_date = "[" + data_name + "]  " + date1;  ax1.set_title(title_date)
#
ax1.legend(loc='upper right', fontsize=8);  
ax4.legend(loc='upper right', fontsize=8);  ax5.legend(loc='upper right', fontsize=8);   ax6.legend(loc='upper right', fontsize=8)

# range
"""
xlim=[1388, 1400]
ax1.set_xlim(xlim);  ax4.set_xlim(xlim);  ax5.set_xlim(xlim);  ax6.set_xlim(xlim)
"""
ylim=[10**(p_min), 10**(p_max)]
ax1.set_ylim(ylim);  ax4.set_ylim(ylim);  ax5.set_ylim(ylim);  ax6.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec-last.png'
    fig.savefig(png_fname)

# Peak
print("[Last]")
peak_E = np.ravel(spec.EE[n_sweep][0]);   peak_f = np.argmax(peak_E); print("Peak EE  :", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(spec.freq[peak_f]), "kHz")
peak_E = np.ravel(spec.EuEu[n_sweep][0]); peak_f = np.argmax(peak_E); print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(spec.freq[peak_f]), "kHz")
peak_E = np.ravel(spec.EvEv[n_sweep][0]); peak_f = np.argmax(peak_E); print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(spec.freq[peak_f]), "kHz")
peak_E = np.ravel(spec.EwEw[n_sweep][0]); peak_f = np.argmax(peak_E); print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(spec.freq[peak_f]), "kHz")

# Frequency - Time diagram

In [ ]:
p_min = p_min0 - 2
p_max = p_max0 - 4
#
n_num = n_time0 * n_block0
EE_2d   = np.ravel(spec.EE);   EE_2d   = EE_2d.reshape  (n_num, n_length0).transpose()
EuEu_2d = np.ravel(spec.EuEu); EuEu_2d = EuEu_2d.reshape(n_num, n_length0).transpose()
EvEv_2d = np.ravel(spec.EvEv); EvEv_2d = EvEv_2d.reshape(n_num, n_length0).transpose()
EwEw_2d = np.ravel(spec.EwEw); EwEw_2d = EwEw_2d.reshape(n_num, n_length0).transpose()
num_1d = np.arange(n_num)

fig2d = plt.figure(figsize=[12,12])
ax1 = fig2d.add_subplot(4, 1, 1);  ax2 = fig2d.add_subplot(4, 1, 2); ax3 = fig2d.add_subplot(4, 1, 3); ax4 = fig2d.add_subplot(4, 1, 4)

# Y-axis
ax1.set_ylim(f_min, f_max); ax2.set_ylim(f_min, f_max); ax3.set_ylim(f_min, f_max); ax4.set_ylim(f_min, f_max)
ax1.set_ylabel('f [kHz]');  ax2.set_ylabel('f [kHz]');  ax3.set_ylabel('f [kHz]');  ax4.set_ylabel('f [kHz]')

# X-axis
ax1.set_title("[" + data_name + "]  " + 'EE');  ax2.set_title('EuEu');  ax3.set_title('EvEv'); ax4.set_title('EwEw')
ax4.set_xlabel(str_date)

# Plot
p1 = ax1.pcolormesh(num_1d, spec.freq, EE_2d[:],   norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
p2 = ax2.pcolormesh(num_1d, spec.freq, EuEu_2d[:], norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
p3 = ax3.pcolormesh(num_1d, spec.freq, EvEv_2d[:], norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
p4 = ax4.pcolormesh(num_1d, spec.freq, EwEw_2d[:], norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')

pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical"); pp1.set_label(power_str)
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical"); pp2.set_label(power_str)
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical"); pp3.set_label(power_str)
pp4 = fig2d.colorbar(p4, ax=ax4, orientation="vertical"); pp4.set_label(power_str)

# Plot
plt.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec-FT.png'
    fig2d.savefig(png_fname)

In [ ]:
p_min = p_min0 - 2
p_max = p_max0 - 4
#
n_num = n_time0 * n_block0
E_DoLuv_2d = np.ravel(spec.E_DoLuv); E_DoLuv_2d = E_DoLuv_2d.reshape(n_num, n_length0).transpose()
E_DoLvw_2d = np.ravel(spec.E_DoLvw); E_DoLvw_2d = E_DoLvw_2d.reshape(n_num, n_length0).transpose()
E_DoLwu_2d = np.ravel(spec.E_DoLwu); E_DoLwu_2d = E_DoLwu_2d.reshape(n_num, n_length0).transpose()
E_DoCuv_2d = np.ravel(spec.E_DoCuv); E_DoCuv_2d = E_DoCuv_2d.reshape(n_num, n_length0).transpose()
E_DoCvw_2d = np.ravel(spec.E_DoCvw); E_DoCvw_2d = E_DoCvw_2d.reshape(n_num, n_length0).transpose()
E_DoCwu_2d = np.ravel(spec.E_DoCwu); E_DoCwu_2d = E_DoCwu_2d.reshape(n_num, n_length0).transpose()
E_ANGuv_2d = np.ravel(spec.E_ANGuv); E_ANGuv_2d = E_ANGuv_2d.reshape(n_num, n_length0).transpose()
E_ANGvw_2d = np.ravel(spec.E_ANGvw); E_ANGvw_2d = E_ANGvw_2d.reshape(n_num, n_length0).transpose()
E_ANGwu_2d = np.ravel(spec.E_ANGwu); E_ANGwu_2d = E_ANGwu_2d.reshape(n_num, n_length0).transpose()
num_1d = np.arange(n_num)

fig2d = plt.figure(figsize=[12,33])
ax1 = fig2d.add_subplot(10, 1, 1);  ax2 = fig2d.add_subplot(10, 1, 2);  ax3 = fig2d.add_subplot(10, 1, 3)
ax4 = fig2d.add_subplot(10, 1, 4);  ax5 = fig2d.add_subplot(10, 1, 5);  ax6 = fig2d.add_subplot(10, 1, 6)
ax7 = fig2d.add_subplot(10, 1, 7);  ax8 = fig2d.add_subplot(10, 1, 8);  ax9 = fig2d.add_subplot(10, 1, 9)

# Y-axis
ax1.set_ylim(f_min, f_max); ax2.set_ylim(f_min, f_max); ax3.set_ylim(f_min, f_max)
ax4.set_ylim(f_min, f_max); ax5.set_ylim(f_min, f_max); ax6.set_ylim(f_min, f_max)
ax7.set_ylim(f_min, f_max); ax8.set_ylim(f_min, f_max); ax9.set_ylim(f_min, f_max)
ax1.set_ylabel('Frequency [kHz]'); ax2.set_ylabel('Frequency [kHz]'); ax3.set_ylabel('Frequency [kHz]')
ax4.set_ylabel('Frequency [kHz]'); ax5.set_ylabel('Frequency [kHz]'); ax6.set_ylabel('Frequency [kHz]')
ax7.set_ylabel('Frequency [kHz]'); ax8.set_ylabel('Frequency [kHz]'); ax9.set_ylabel('Frequency [kHz]')

# X-axis
ax1.set_title("[" + data_name + "]  " + 'Deg. Lin. uv');     ax2.set_title('Deg. Lin. vw');     ax3.set_title('Deg. Lin. wu')
ax4.set_title('Deg. Cir. uv');     ax5.set_title('Deg. Cir. vw');     ax6.set_title('Deg. Cir. wu')
ax7.set_title('Ang. uv');          ax8.set_title('Ang. vw');          ax9.set_title('Ang. wu')
ax9.set_xlabel(str_date)

# Plot
p1 = ax1.pcolormesh(num_1d, spec.freq, E_DoLuv_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
p2 = ax2.pcolormesh(num_1d, spec.freq, E_DoLvw_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
p3 = ax3.pcolormesh(num_1d, spec.freq, E_DoLwu_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
p4 = ax4.pcolormesh(num_1d, spec.freq, E_DoCuv_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
p5 = ax5.pcolormesh(num_1d, spec.freq, E_DoCvw_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
p6 = ax6.pcolormesh(num_1d, spec.freq, E_DoCwu_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
p7 = ax7.pcolormesh(num_1d, spec.freq, E_ANGuv_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
p8 = ax8.pcolormesh(num_1d, spec.freq, E_ANGvw_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
p9 = ax9.pcolormesh(num_1d, spec.freq, E_ANGwu_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')

pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical")
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical")
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical")
pp4 = fig2d.colorbar(p4, ax=ax4, orientation="vertical")
pp5 = fig2d.colorbar(p5, ax=ax5, orientation="vertical")
pp6 = fig2d.colorbar(p6, ax=ax6, orientation="vertical")
pp7 = fig2d.colorbar(p7, ax=ax7, orientation="vertical")
pp8 = fig2d.colorbar(p8, ax=ax8, orientation="vertical")
pp9 = fig2d.colorbar(p9, ax=ax9, orientation="vertical")

# Plot
plt.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_AngT-FT.png'
    fig2d.savefig(png_fname)

# Get Auto Correlation

In [ ]:
auto = juice_sid23.hf_sid23_getauto(data)

print(auto.EuEu.shape, n_time0, n_block0, n_length0)
print(f_min, "kHz -", f_max, "kHz  (Center:", data.freq_center[0], "kHz  df:", d_freq, "kHz")

## Auto First

In [ ]:
n_sweep = 0

fig = plt.figure(figsize=(12, 10))
ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4)

ax1.plot(data.time[0][0], np.ravel(auto.EE[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax2.plot(data.time[0][0], np.ravel(auto.EuEu[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax3.plot(data.time[0][0], np.ravel(auto.EvEv[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax4.plot(data.time[0][0], np.ravel(auto.EwEw[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax1.plot(data.time[0][0], np.ravel(auto.EE[n_sweep][np.int16(n_block/2)][:]), '-g', linewidth=.5, label='half')
ax2.plot(data.time[0][0], np.ravel(auto.EuEu[n_sweep][np.int16(n_block/2)][:]), '-g', linewidth=.5, label='half')
ax3.plot(data.time[0][0], np.ravel(auto.EvEv[n_sweep][np.int16(n_block/2)][:]), '-g', linewidth=.5, label='half')
ax4.plot(data.time[0][0], np.ravel(auto.EwEw[n_sweep][np.int16(n_block/2)][:]), '-g', linewidth=.5, label='half')
ax1.plot(data.time[0][0], np.ravel(auto.EE[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax2.plot(data.time[0][0], np.ravel(auto.EuEu[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax3.plot(data.time[0][0], np.ravel(auto.EvEv[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax4.plot(data.time[0][0], np.ravel(auto.EwEw[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax1.set_ylabel('EE');  ax2.set_ylabel('EuEu'); ax3.set_ylabel('EvEv'); ax4.set_ylabel('EwEw'); ax6.set_xlabel('Time [sec]')
#
date1 = data.epoch[n_sweep];  date1 = date1.strftime('First: %Y/%m/%d %R:%S')
title_date = "[" + data_name + "]  " + date1;  ax1.set_title(title_date)
#
ax1.legend(loc='upper right', fontsize=8);  
ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8);   ax4.legend(loc='upper right', fontsize=8)

# range: X-axis
# xlim=[0, 0.0001];  ax1.set_xlim(xlim);  ax2.set_xlim(xlim);  ax3.set_xlim(xlim);  ax4.set_xlim(xlim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_auto-1st.png'
    fig.savefig(png_fname)

# Peak
peak_E = np.ravel(auto.EE[n_sweep][0][10:]);  peak_T = np.ravel(data.time[0][0][10:])
peak_t = np.argmax(peak_E); print("Peak EE  :", '{:.2e}'.format(peak_E[peak_t]), "at", '{:.2f}'.format(peak_T[peak_t]*1000), "msec")

## Auto - Mid

In [ ]:
n_sweep = np.int16(n_time0//2)
print("[mid]  SWEEP:", n_num0, "in total of", n_time0)

fig = plt.figure(figsize=(12, 10))
ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4)

ax1.plot(data.time[0][0], np.ravel(auto.EE[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax2.plot(data.time[0][0], np.ravel(auto.EuEu[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax3.plot(data.time[0][0], np.ravel(auto.EvEv[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax4.plot(data.time[0][0], np.ravel(auto.EwEw[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax1.plot(data.time[0][0], np.ravel(auto.EE[n_sweep][np.int16(n_block/2)][:]), '-g', linewidth=.5, label='half')
ax2.plot(data.time[0][0], np.ravel(auto.EuEu[n_sweep][np.int16(n_block/2)][:]), '-g', linewidth=.5, label='half')
ax3.plot(data.time[0][0], np.ravel(auto.EvEv[n_sweep][np.int16(n_block/2)][:]), '-g', linewidth=.5, label='half')
ax4.plot(data.time[0][0], np.ravel(auto.EwEw[n_sweep][np.int16(n_block/2)][:]), '-g', linewidth=.5, label='half')
ax1.plot(data.time[0][0], np.ravel(auto.EE[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax2.plot(data.time[0][0], np.ravel(auto.EuEu[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax3.plot(data.time[0][0], np.ravel(auto.EvEv[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax4.plot(data.time[0][0], np.ravel(auto.EwEw[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax1.set_ylabel('EE');  ax2.set_ylabel('EuEu'); ax3.set_ylabel('EvEv'); ax4.set_ylabel('EwEw'); ax4.set_xlabel('Time [sec]')
#
date1 = data.epoch[n_sweep];  date1 = date1.strftime('Mid: %Y/%m/%d %R:%S')
title_date = "[" + data_name + "]  " + date1;  ax1.set_title(title_date)
#
ax1.legend(loc='upper right', fontsize=8);  
ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8);   ax4.legend(loc='upper right', fontsize=8)

# range: X-axis
# xlim=[0, 0.0001]; ax1.set_xlim(xlim);  ax2.set_xlim(xlim);  ax3.set_xlim(xlim);  ax4.set_xlim(xlim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_auto-mid.png'
    fig.savefig(png_fname)

# Peak
peak_E = np.ravel(auto.EE[n_sweep][0][10:]);  peak_T = np.ravel(data.time[0][0][10:])
peak_t = np.argmax(peak_E); print("Peak EE  :", '{:.2e}'.format(peak_E[peak_t]), "at", '{:.2f}'.format(peak_T[peak_t]*1000), "msec")

## Auto - Last

In [ ]:
n_sweep = n_time0 - 1

fig = plt.figure(figsize=(12, 10))
ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4)

ax1.plot(data.time[0][0], np.ravel(auto.EE[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax2.plot(data.time[0][0], np.ravel(auto.EuEu[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax3.plot(data.time[0][0], np.ravel(auto.EvEv[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax4.plot(data.time[0][0], np.ravel(auto.EwEw[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax1.plot(data.time[0][0], np.ravel(auto.EE[n_sweep][np.int16(n_block/2)][:]), '-g', linewidth=.5, label='half')
ax2.plot(data.time[0][0], np.ravel(auto.EuEu[n_sweep][np.int16(n_block/2)][:]), '-g', linewidth=.5, label='half')
ax3.plot(data.time[0][0], np.ravel(auto.EvEv[n_sweep][np.int16(n_block/2)][:]), '-g', linewidth=.5, label='half')
ax4.plot(data.time[0][0], np.ravel(auto.EwEw[n_sweep][np.int16(n_block/2)][:]), '-g', linewidth=.5, label='half')
ax1.plot(data.time[0][0], np.ravel(auto.EE[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax2.plot(data.time[0][0], np.ravel(auto.EuEu[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax3.plot(data.time[0][0], np.ravel(auto.EvEv[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax4.plot(data.time[0][0], np.ravel(auto.EwEw[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax1.set_ylabel('EE');  ax2.set_ylabel('EuEu'); ax3.set_ylabel('EvEv'); ax4.set_ylabel('EwEw'); ax4.set_xlabel('Time [sec]')
#
date1 = data.epoch[n_sweep];  date1 = date1.strftime('Last: %Y/%m/%d %R:%S')
title_date = "[" + data_name + "]  " + date1;  ax1.set_title(title_date)
#
ax1.legend(loc='upper right', fontsize=8);  
ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8);   ax4.legend(loc='upper right', fontsize=8)

# range: X-axis
# xlim=[0, 0.0001]; ax1.set_xlim(xlim);  ax2.set_xlim(xlim);  ax3.set_xlim(xlim);  ax4.set_xlim(xlim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_auto-last.png'
    fig.savefig(png_fname)

# Peak
peak_E = np.ravel(auto.EE[n_sweep][0][10:]);  peak_T = np.ravel(data.time[0][0][10:])
peak_t = np.argmax(peak_E); print("Peak EE  :", '{:.2e}'.format(peak_E[peak_t]), "at", '{:.2f}'.format(peak_T[peak_t]*1000), "msec")

## Auto - Time

In [28]:
n_num = n_time0 * n_block0
p_min = -1/4
p_max =  1/4
EE_2d   = np.ravel(auto.EE);    EE_2d   = EE_2d.reshape(n_num, n_length0).transpose()
EuEu_2d = np.ravel(auto.EuEu);  EuEu_2d = EuEu_2d.reshape(n_num, n_length0).transpose()
EvEv_2d = np.ravel(auto.EvEv);  EvEv_2d = EvEv_2d.reshape(n_num, n_length0).transpose()
EwEw_2d = np.ravel(auto.EwEw);  EwEw_2d = EwEw_2d.reshape(n_num, n_length0).transpose()
num_1d = np.arange(n_num)

fig2d = plt.figure(figsize=[12,12])
ax1 = fig2d.add_subplot(4, 1, 1);  ax2 = fig2d.add_subplot(4, 1, 2);  ax3 = fig2d.add_subplot(4, 1, 3);  ax4 = fig2d.add_subplot(4, 1, 4)

# Y-axis
ax1.set_ylabel('time [sec]');  ax2.set_ylabel('time [sec]');  ax3.set_ylabel('time [sec]');  ax4.set_ylabel('time [sec]')

# X-axis
ax1.set_title("[" + data_name + "]  " + 'EE');  ax2.set_title('EuEu');  ax3.set_title('EvEv'); ax4.set_title('EwEw')
ax4.set_xlabel(str_date)

# Plot
p1 = ax1.pcolormesh(num_1d, data.time[0][0], EE_2d[:],   vmin=p_min, vmax=p_max, cmap='bwr')
p2 = ax2.pcolormesh(num_1d, data.time[0][0], EuEu_2d[:], vmin=p_min, vmax=p_max, cmap='bwr')
p3 = ax3.pcolormesh(num_1d, data.time[0][0], EvEv_2d[:], vmin=p_min, vmax=p_max, cmap='bwr')
p4 = ax4.pcolormesh(num_1d, data.time[0][0], EwEw_2d[:], vmin=p_min, vmax=p_max, cmap='bwr')

pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical");  pp1.set_label('auto-corr')
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical");  pp2.set_label('auto-corr')
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical");  pp3.set_label('auto-corr')
pp4 = fig2d.colorbar(p4, ax=ax4, orientation="vertical");  pp4.set_label('auto-corr')

# Plot
plt.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_auto-FT.png'
    fig2d.savefig(png_fname)